In [ ]:
import requests
import os
from dotenv import load_dotenv, set_key
from strava_client.strava import Strava

load_dotenv()

In [ ]:
import requests

CLIENT_ID = "YOUR_CLIENT_ID"
CLIENT_SECRET = "YOUR_CLIENT_SECRET"
CODE = "CODE_FROM_STEP_1"  # the code you got from the redirect

token_url = "https://www.strava.com/oauth/token"

data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "code": CODE,
    "grant_type": "authorization_code"
}

response = requests.post(token_url, data=data)
token_info = response.json()

access_token = token_info["access_token"]
refresh_token = token_info["refresh_token"]
expires_at = token_info["expires_at"]

print("Access token:", access_token)
print("Refresh token:", refresh_token)
print("Expires at:", expires_at)


In [ ]:
from datetime import datetime, timedelta

datetime.now() - timedelta(days=1)

In [ ]:
import stravalib
from datetime import datetime

load_dotenv()

STRAVA_CLIENT_ID = os.getenv("STRAVA_CLIENT_ID")
STRAVA_CLIENT_SECRET = os.getenv("STRAVA_CLIENT_SECRET")

client = stravalib.Client()
client.authorization_url(
    client_id=STRAVA_CLIENT_ID,
    redirect_uri="http://localhost",
    scope=['read', 'read_all', 'profile:read_all', 'profile:write', 'activity:read', 'activity:read_all', 'activity:write', 'activity:read_permission']
)

In [ ]:
response = client.exchange_code_for_token(STRAVA_CLIENT_ID, STRAVA_CLIENT_SECRET, "XXXXXXX")
print(response)

for key, value in {
    "STRAVA_ACCESS_TOKEN": response["access_token"],
    "STRAVA_REFRESH_TOKEN": response["refresh_token"],
    "STRAVA_TOKEN_EXPIRES_AT": str(response["expires_at"])
}.items():
    set_key(".env", key, value)

In [ ]:
load_dotenv(override=True)
STRAVA_ACCESS_TOKEN = os.getenv("STRAVA_ACCESS_TOKEN")
STRAVA_REFRESH_TOKEN = os.getenv("STRAVA_REFRESH_TOKEN")
STRAVA_TOKEN_EXPIRES_AT = float(os.getenv("STRAVA_TOKEN_EXPIRES_AT"))
client = stravalib.Client(access_token=STRAVA_ACCESS_TOKEN, refresh_token=STRAVA_REFRESH_TOKEN, token_expires=STRAVA_TOKEN_EXPIRES_AT)

bfr = datetime.now() + timedelta(days=1)
aftr = datetime.now() - timedelta(days=2)

result = client.get_activities(before=bfr, after=aftr)
[print(r) for r in result]

In [ ]:
from util.strava_client_interface import StravaWrapper
from datetime import datetime, timedelta

bfr = datetime.now() + timedelta(days=1)
aftr = datetime.now() - timedelta(days=2)

res = StravaWrapper().get_activities(before=bfr, after=aftr)

[print(r) for r in res]

res.next().dict()

## Bronze data exploration

In [2]:
from util.db_connection import PsycopgConnection
import pandas as pd

In [3]:
conn = PsycopgConnection().conn
sql = "SELECT * FROM bronze_all_activity"
df = pd.read_sql(sql, conn)
df.head()

[10/10/25 11:11:04] INFO     2025-10-10 11:11:04,283 [INFO] strava-analysis: Creating db        ]8;id=682688;file:///Users/luiscruz/Desktop/strava-analysis/util/db_connection.py\db_connection.py]8;;\:]8;id=330757;file:///Users/luiscruz/Desktop/strava-analysis/util/db_connection.py#12\12]8;;\
                             connection object                                                                     

                    WARNING  2025-10-10 11:11:04,936 [WARNING] py.warnings:                         ]8;id=519149;file:///Users/luiscruz/.local/share/uv/python/cpython-3.13.3-macos-x86_64-none/lib/python3.13/warnings.py\warnings.py]8;;\:]8;id=547871;file:///Users/luiscruz/.local/share/uv/python/cpython-3.13.3-macos-x86_64-none/lib/python3.13/warnings.py#110\110]8;;\
                             /var/folders/jq/tdq0h8rj1zdbxsbs18vhfp0h0000gp/T/ipykernel_80008/23490                
                             79700.py:3: UserWarning: pandas only supports SQLAlchemy connectable                  
                             (engine/connection) or database string URI or sqlite3 DBAPI2                          
                             connection. Other DBAPI2 objects are not tested. Please consider using                
                             SQLAlchemy.                                                                           
                               df = pd.read_sql(sql, conn)                                                         
                                                                                                                   

,id,achievement_count,athlete,athlete_count,average_speed,average_watts,comment_count,commute,device_watts,distance,...,location_country,pr_count,suffer_score,has_heartrate,average_heartrate,max_heartrate,average_cadence,from_accepted_tag,visibility,ingested_at
0,16093491911,0,"{'id': 105333117, 'resource_state': 1}",1,7.254,117.4,0,False,True,25048.2,...,None,0,NaN,False,NaN,NaN,67.8,False,everyone,2025-10-10 13:29:01.299150+00:00
1,16082941463,1,"{'id': 105333117, 'resource_state': 1}",1,2.301,NaN,0,False,None,5013.3,...,None,0,64.0,True,165.2,180.0,81.6,False,everyone,2025-10-10 13:29:01.299150+00:00
2,16072362664,0,"{'id': 105333117, 'resource_state': 1}",1,7.537,129.7,0,False,True,45230.7,...,None,0,NaN,False,NaN,NaN,63.2,False,everyone,2025-10-10 13:29:01.299150+00:00
3,16060416843,1,"{'id': 105333117, 'resource_state': 1}",1,2.074,NaN,0,False,None,5929.9,...,None,0,50.0,True,155.3,172.0,80.3,False,everyone,2025-10-10 13:29:01.299150+00:00
4,16049493185,0,"{'id': 105333117, 'resource_state': 1}",1,7.372,121.1,0,False,True,30009.4,...,None,0,NaN,False,NaN,NaN,72.8,False,everyone,2025-10-10 13:29:01.299150+00:00
